In [1]:
import os
import numpy as np
import pandas as pd
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
data_dir = '/Users/saikarthik/Downloads/untitled folder/parkinsons_dataset'


In [3]:
img_height, img_width = 224, 224 
batch_size = 32
epochs = 20  


In [4]:
datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    validation_split=0.2,  
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
)


In [5]:

train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    subset='training',
)

Found 665 images belonging to 2 classes.


In [6]:

validation_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    subset='validation',
)

Found 166 images belonging to 2 classes.


In [7]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))


2025-04-02 10:53:27.069857: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Pro
2025-04-02 10:53:27.069885: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-04-02 10:53:27.069896: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2025-04-02 10:53:27.069929: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-04-02 10:53:27.069945: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [8]:
for layer in base_model.layers:
    layer.trainable = False

model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])


In [9]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

In [10]:
# Train the model
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=50,
)

Epoch 1/50


2025-04-02 10:53:28.684028: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


21/21 [==============================] - ETA: 0s - loss: 0.6388 - accuracy: 0.6857

2025-04-02 10:53:36.582092: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


21/21 [==============================] - 10s 316ms/step - loss: 0.6388 - accuracy: 0.6857 - val_loss: 0.5889 - val_accuracy: 0.7349
Epoch 2/50
21/21 [==============================] - 6s 263ms/step - loss: 0.6170 - accuracy: 0.7143 - val_loss: 0.5709 - val_accuracy: 0.7349
Epoch 3/50
21/21 [==============================] - 6s 271ms/step - loss: 0.6148 - accuracy: 0.7113 - val_loss: 0.5582 - val_accuracy: 0.7349
Epoch 4/50
21/21 [==============================] - 6s 271ms/step - loss: 0.5994 - accuracy: 0.7158 - val_loss: 0.5453 - val_accuracy: 0.7349
Epoch 5/50
21/21 [==============================] - 6s 263ms/step - loss: 0.5694 - accuracy: 0.7444 - val_loss: 0.5415 - val_accuracy: 0.7349
Epoch 6/50
21/21 [==============================] - 6s 267ms/step - loss: 0.5412 - accuracy: 0.7474 - val_loss: 0.5333 - val_accuracy: 0.7349
Epoch 7/50
21/21 [==============================] - 6s 261ms/step - loss: 0.5544 - accuracy: 0.7338 - val_loss: 0.5193 - val_accuracy: 0.7349
Epoch 8/50
21/21

In [11]:
val_loss, val_accuracy = model.evaluate(validation_generator)
print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")

validation_generator.reset()
predictions = (model.predict(validation_generator) > 0.5).astype("int64")
true_classes = validation_generator.classes

print("\nClassification Report:")
print(classification_report(true_classes, predictions))

print("\nConfusion Matrix:")
print(confusion_matrix(true_classes, predictions))

6/6 [==============================] - 1s 168ms/step - loss: 0.3861 - accuracy: 0.7952
Validation Accuracy: 79.52%


2025-04-02 10:58:14.456897: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


6/6 [==============================] - 1s 196ms/step

Classification Report:
              precision    recall  f1-score   support

           0       0.73      0.92      0.81       122
           1       0.23      0.07      0.11        44

    accuracy                           0.69       166
   macro avg       0.48      0.49      0.46       166
weighted avg       0.60      0.69      0.63       166


Confusion Matrix:
[[112  10]
 [ 41   3]]
